<a href="https://colab.research.google.com/github/rcl0511/OPENSOURCECOLABO/blob/tts/soskin_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sroberta-multitask')
!pip install flask pyngrok sentence-transformers pandas torch
!ngrok config add-authtoken &lt;your_personal_ngrok_auth_token&gt;

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
import os
import re
import io
import torch
import torch.nn as nn
import pandas as pd
from flask import Flask, request, jsonify
from pyngrok import ngrok
from PIL import Image
from sentence_transformers import SentenceTransformer, util
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# 🔹 모델 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
text_model = SentenceTransformer('jhgan/ko-sroberta-multitask', device=device)

# 🔹 데이터 불러오기
def load_answer_csv(path):
    try:
        return pd.read_csv(path, encoding='cp949')
    except:
        return pd.read_csv(path, encoding='utf-8')

df_q = pd.read_pickle('/content/drive/MyDrive/오픈소스/화상_질문_with_embedding.pkl')
df_q['embedding'] = df_q['embedding'].apply(lambda x: torch.tensor(x).to(device))
df_a = load_answer_csv('/content/drive/MyDrive/오픈소스/화상_답변.csv')

# 🔹 질환_의도 키 생성 (⚠️ 여기 추가됨!)
for df in [df_q, df_a]:
    df['fileName'] = df['fileName'].astype(str).str.strip().str.upper()
    df['질환_의도'] = df['disease_name'].astype(str).str.strip() + "_" + df['intention'].astype(str).str.strip()
df_a['answer'] = df_a['answer'].fillna('')

# 🔹 텍스트 처리 함수
def preprocess(text):
    return re.sub(r"[^\w\s]", "", text).strip().lower()

def find_best_conditions(user_input, top_k=3):
    input_embedding = text_model.encode(preprocess(user_input), convert_to_tensor=True).to(device)
    similarities = [util.cos_sim(qe, input_embedding).item() for qe in df_q['embedding']]
    top_indices = torch.topk(torch.tensor(similarities), top_k).indices.tolist()
    result = [{
        "question": df_q.iloc[idx]['question'],
        "condition_key": df_q.iloc[idx]['질환_의도'],
        "similarity": round(similarities[idx], 4)
    } for idx in top_indices]
    return result, input_embedding

def get_best_answer(user_input):
    user_input = user_input.strip()
    if not user_input:
        return {"answer": "❌ 입력된 내용이 없어요.", "matches": [], "top_similar_questions": []}

    similar_info, input_embedding = find_best_conditions(user_input)
    condition_key = similar_info[0]['condition_key']
    answer_df = df_a[df_a['질환_의도'] == condition_key]

    if answer_df.empty:
        return {"answer": "❌ 해당 주제에 대한 답변이 없습니다.", "matches": similar_info, "top_similar_questions": similar_info}

    priority_keywords = ['조치', '예방', '대응', '응급', '처치', '초기', '해결', '냉찜질', '물로 식히기', '연고', '병원']
    filtered_df = answer_df[answer_df['answer'].str.contains('|'.join(priority_keywords), na=False)]
    target_df = filtered_df if not filtered_df.empty else answer_df

    answer_texts = target_df['answer'].fillna('').tolist()
    answer_embeddings = text_model.encode(answer_texts, convert_to_tensor=True).to(device)
    similarities = util.cos_sim(input_embedding, answer_embeddings)[0]
    best_idx = torch.argmax(similarities).item()

    return {
        "answer": answer_texts[best_idx].strip() + " 더 궁금한 점이 있으신가요?",
        "matches": similar_info,
        "top_similar_questions": similar_info
    }

# 🔹 이미지 모델 설정
def get_image_model(num_classes=3):
    weights = EfficientNet_B0_Weights.DEFAULT
    model = efficientnet_b0(weights=weights)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    return model

image_model = get_image_model()
model_path = "/content/drive/MyDrive/오픈소스/best_model.pt"
image_model.load_state_dict(torch.load(model_path, map_location=device))
image_model.eval().to(device)

def predict_image(image_file):
    image = Image.open(image_file).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = image_model(tensor)
        _, predicted = torch.max(output, 1)

    return int(predicted.item())

# 🔹 Flask 서버 정의
app = Flask(__name__)

@app.route("/answer", methods=["POST"])
def answer_api():
    data = request.get_json()
    user_input = data.get("keyword", "")
    result = get_best_answer(user_input)
    return jsonify(result)

@app.route("/predict-image", methods=["POST"])
def predict_image_api():
    if 'file' not in request.files:
        return jsonify({"error": "이미지 파일이 필요합니다"}), 400

    file = request.files['file']
    try:
        prediction = predict_image(file)
        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# 🔹 ngrok 실행
public_url = ngrok.connect(5000)
print("✅ 서버 주소:", public_url)
app.run(port=5000)


✅ 서버 주소: NgrokTunnel: "https://c66a-34-87-87-92.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 12:46:22] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 12:52:06] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 13:02:15] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 13:18:57] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 13:22:06] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 13:24:04] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 13:25:11] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 13:37:06] "POST /answer HTTP/1.1" 200 -
